__собираем предложения по квартирам__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/flat/avito' 
cian_raw_data_path = 'data/raw/flat/cian'

result_file_path = 'data/data_flat.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/flat/avito/avito_2022-08-26_17-29_raw.xlsx',
 'data/raw/flat/avito/avito_2022-08-29_13-16_raw.xlsx',
 'data/raw/flat/avito/avito_2022-08-31_17-42_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-01_14-07_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-02_13-04_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-05_14-02_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-06_13-09_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-07_12-39_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-08_13-36_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-12_13-44_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-13_13-55_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-15_10-27_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-16_15-07_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-19_12-36_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-22_09-48_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-26_11-20_raw.xlsx']

In [7]:
from lib.avito import AvitoDataCleanerRealtyFlat

data_avito = pd.concat([
        pd.read_excel(f)
        .dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

data_avito = AvitoDataCleanerRealtyFlat().transform( data_avito )

print(len(data_avito))

20271


In [8]:
assert len(data_avito)>0

In [9]:
data_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20271 entries, 0 to 20270
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   avito_id       20271 non-null  int64         
 1   title          20271 non-null  object        
 2   price          20271 non-null  int64         
 3   adr            20271 non-null  object        
 4   description    20271 non-null  object        
 5   obj_name       20271 non-null  object        
 6   ts             20271 non-null  datetime64[ns]
 7   avito_page     20271 non-null  int64         
 8   place          20271 non-null  object        
 9   nrooms         20271 non-null  int64         
 10  floor          20271 non-null  int64         
 11  nfloors        20271 non-null  int64         
 12  area           20271 non-null  float64       
 13  is_studio      20271 non-null  bool          
 14  is_apartment   20271 non-null  bool          
 15  is_part        2027

In [10]:
data_avito.isnull().any()

avito_id         False
title            False
price            False
adr              False
description      False
obj_name         False
ts               False
avito_page       False
place            False
nrooms           False
floor            False
nfloors          False
area             False
is_studio        False
is_apartment     False
is_part          False
is_auction       False
is_openspace     False
is_roof          False
is_SNT           False
priceM           False
is_last_floor    False
dtype: bool

In [11]:
# data_avito.query('nrooms==0')
# data_avito[ data_avito['title'].str.len()<1]

## собираем данные циан.ру

In [12]:
import re
from os import listdir

raw_data_files = sorted([
        cian_raw_data_path+'/'+f 
        for f in listdir(cian_raw_data_path) 
        if re.match(r'.+\.pkl$',f)
        #if re.match(r'.+\.xlsx$',f)
    
    ])
raw_data_files

['data/raw/flat/cian/cian_2022-09-08_14-25_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-12_13-59_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-13_13-59_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-15_10-30_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-16_15-11_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-19_12-40_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-22_09-53_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-26_11-23_raw.pkl']

In [13]:
data_cian = pd.concat([
        pd.read_pickle(f)
         .dropna()
         .drop_duplicates(['LinkArea','Description',])
         .rename(columns={'page':'cian_page'})
         .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

In [14]:
data_cian.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11943 entries, 0 to 11942
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   OfferTitle     11943 non-null  object        
 1   OfferSubtitle  11943 non-null  object        
 2   Deadline       11943 non-null  object        
 3   MainPrice      11943 non-null  object        
 4   PriceInfo      11943 non-null  object        
 5   GeoLabel       11943 non-null  object        
 6   TimeLabel      11943 non-null  object        
 7   LinkArea       11943 non-null  object        
 8   Description    11943 non-null  object        
 9   cian_page      11943 non-null  int64         
 10  ts             11943 non-null  datetime64[ns]
 11  place          11943 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 1.1+ MB


In [15]:
data_cian.isnull().any()

OfferTitle       False
OfferSubtitle    False
Deadline         False
MainPrice        False
PriceInfo        False
GeoLabel         False
TimeLabel        False
LinkArea         False
Description      False
cian_page        False
ts               False
place            False
dtype: bool

In [16]:
swap_title = data_cian['OfferSubtitle'].str.match(r'.*\d+ м², \d+/\d+ эт.*') 

data_cian = pd.concat([
        data_cian[ ~swap_title ],
        data_cian[ swap_title ]
            .rename(columns={'OfferTitle':'OfferSubtitle','OfferSubtitle':'OfferTitle'})
    ]).reset_index(drop=True)


In [17]:
# data_cian

In [18]:
from lib.cian import CianDataCleaner

data_cian = CianDataCleaner().transform( data_cian )

print(len(data_cian))

11943


In [19]:
assert len(data_cian)>0

In [20]:
data_cian.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11943 entries, 0 to 11942
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          11943 non-null  object        
 1   obj_name       11943 non-null  object        
 2   adr            11943 non-null  object        
 3   nrooms         11943 non-null  int64         
 4   floor          11943 non-null  int64         
 5   nfloors        11943 non-null  int64         
 6   area           11943 non-null  float64       
 7   is_studio      11943 non-null  bool          
 8   is_apartment   11943 non-null  bool          
 9   is_part        11943 non-null  bool          
 10  is_auction     11943 non-null  bool          
 11  is_openspace   11943 non-null  bool          
 12  is_roof        11943 non-null  bool          
 13  is_SNT         11943 non-null  bool          
 14  price          11943 non-null  int64         
 15  priceM         1194

In [21]:
data_cian.isnull().any()

title            False
obj_name         False
adr              False
nrooms           False
floor            False
nfloors          False
area             False
is_studio        False
is_apartment     False
is_part          False
is_auction       False
is_openspace     False
is_roof          False
is_SNT           False
price            False
priceM           False
is_last_floor    False
cian_url         False
description      False
cian_page        False
ts               False
place            False
dtype: bool

In [22]:
# data_cian
# data_cian.query('nrooms==0')
# data_cian[ data_cian['title'].str.len()<1 ]
# data_cian[ data_cian['obj_name'].str.len()>0 ][['title','obj_name',]] # .to_csv('tmp/cian_title.csv',sep='\t')|

----

In [23]:
data_avito['url'] = 'https://avito.ru/' + data_avito['avito_id'].astype(int).astype(str)
data_avito = data_avito.drop(columns=['avito_id'])
data_cian = data_cian.rename(columns={'cian_url':'url'})

In [24]:
data = pd.concat([data_avito,data_cian]).reset_index(drop=True) 
print(len(data))
assert len(data)>0

32214


In [25]:
data.sample()

,title,price,adr,description,obj_name,ts,avito_page,place,nrooms,floor,...,is_apartment,is_part,is_auction,is_openspace,is_roof,is_SNT,priceM,is_last_floor,url,cian_page
27129,"2-комн. кв., 62 м², 5/10 этаж",9400000,"Севастополь, район Гагаринский, улица Братьев Манганари, 26 к1","Двухкомнатная квартира 60,2 м2 на 5/10 этаже дома с просторной кухней 10 м2. Расположена в г. Севастополь, Гагаринский район, ул. Братьев Манганари, д. 26, к1.\nПозвоните нам, чтобы узнать все детали! \n \nТехнические характеристики: \n \n-Адрес: г. Севастополь, Гагаринский район, ул. Братьев Манганари, д. 26\n-Год постройки дома: 2021\n-Этаж /этажность: 5/10\n-Количество комнат: 2\n-Комнаты: изолированные\n-Общая площадь: м2\n-Жилая площадь: м2\n-Комната: 16,4; 17,2 м2\n-Площадь кухни: 10 м2\n-Высота потолка: 2,55 м\n-Санузел: раздельный\n-Водоснабжение: центральное\n-Горячее водоснабжение: АГВ\n-Отопление: АГВ\nДополнительная информация: \nОбщая площадь квартиры 60.2 кв.м. Комнаты изолированные, на 2 стороны, 16.4 и 17.2 кв.м. Просторная кухня 10 кв.м, санузел раздельный. Ванная комната 6 кв.м. с окном. \nВ квартире установлены двойные стеклопакеты, разведена проводка, розетки, выключатели, радиаторы, все счетчики, двухконтурный газовый котел (АГВ). \nБлагоустройство дворовой территории выполнено по всем современным стандартам: огороженные детские площадки с безопасным покрытием, оборудованные площадки для спорта\nПозвоните нам, чтобы назначить просмотр! \nИнфраструктура: \nНовый детский сад, школы образовательного центра ""Казачья бухта"", детская поликлиника, рынок, а самое главное - море в пешей доступности! До пляжа всего 300 метров! Пляж не городской, его посещают только жители Порто-Франко. \nЛюбой вид расчёта.\nДокументы РФ, проверены юристами и готовы к сделке. \nПрофессиональное сопровождение до получения права собственности.\nДобавьте предложение в закладки, чтобы не потерять!",,2022-09-16 15:11:49.899057,NaN,sevastopol,2,5,...,False,False,False,False,False,False,9.40,False,https://sevastopol.cian.ru/sale/flat/273530099/,42.00


## обновляем таблицу адресов

In [26]:
# data_ = data.sample(15).reset_index(drop=True)
# data_[['adr']]

In [27]:
places = { 
# поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
  'sevastopol': 'Севастополь,',
}

def norm_place(d,places=places):
    if not d['place'] in places: return d['adr_']
    p = places[ d['place'] ]
    return d['adr_'] if d['adr_'].find(p)>-1 else p+d['adr_']

In [28]:
from lib.locator import AddressTransformer

atr = AddressTransformer()
data['adr_'] = data['adr'].progress_apply(atr.transform)
data['adr_'] = data[['adr_','place']].progress_apply(norm_place,axis=1)
data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

  0%|          | 0/32214 [00:00<?, ?it/s]

  0%|          | 0/32214 [00:00<?, ?it/s]

In [29]:
data[['adr_orig','adr']].sample(10)

,adr_orig,adr
4749,"Нахимовский муниципальный округ, посёлок Любимовка, улица Софьи Перовской, 52","Севастополь,Нахимовский район , Любимовка, улица Софьи Перовской, 52"
31365,"Севастополь, район Нахимовский, микрорайон Бартеньевка, улица Богданова, 10А","Севастополь, район Нахимовский, Бартеньевка, улица Богданова, 10А"
15138,"аллея Защитников 35-й Батареи, д. 11а","Севастополь,аллея Защитников 35-й Батареи, д. 11а"
143,"улица Горпищенко, 94","Севастополь, улица Горпищенко, 94"
27675,"Севастополь, район Гагаринский, Казачья Бухта мкр, Скифия ЖК","Севастополь, район Гагаринский, Казачья Бухта мкр, Скифия ЖК"
15954,"улица Лётчиков, д. 10","Севастополь, улица Лётчиков, д. 10"
25605,"Севастополь, район Ленинский, улица Новороссийская, 57","Севастополь, район Ленинский, улица Новороссийская, 57"
24685,"Севастополь, район Ленинский, микрорайон Острякова, улица Николая Музыки, 54","Севастополь, район Ленинский, Острякова, улица Николая Музыки, 54"
5171,"Советская улица , 41","Севастополь,Советская улица , 41"
11284,"проспект Генерала Острякова, 244 к5","Севастополь, проспект Генерала Острякова, 244 к5"


In [30]:
# from lib.locator import GeocoderSimpleOSM
from lib.locator import GeocoderYandex
from yandex_keys import keys

atr = AddressTransformer()
locator = GeocoderYandex(keys[0])
# locator = GeocoderSimpleOSM()

loc_file_path = 'data/location.pkl' # таблица адресов
loc = pd.read_pickle(loc_file_path)
# loc = pd.DataFrame([],columns=['adr','latitude','longitude',])
logging.info(f'LocationUpdater: {len(loc)} addresses in index')

# собираем все адреса в один список
loc = loc.merge(
        data[['adr']].drop_duplicates(),
        how='outer',
        on='adr',
    ).drop_duplicates().reset_index(drop=True)

logging.info(f'LocationUpdater: {len(loc)} addresses total')

# собираем все адреса с геопозицией
loc_def = loc[~loc['latitude'].isna()].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_def)} addresses defined')

# собираем все адреса без геопозиции
loc_undef = loc[loc['latitude'].isna()][['adr']].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_undef)} addresses undefined')

if len(loc_undef)>0:
    # определяем координаты по строке адреса
    loc_undef = loc_undef.join( loc_undef['adr'].progress_apply(locator.transform).apply(pd.Series) ).dropna()     
    logging.info(f'LocationUpdater: {len(loc_undef)} new addresses found')
    loc = pd.concat([loc_def,loc_undef]).reset_index(drop=True) 
    assert len(loc)>0
    loc.to_pickle(loc_file_path)

[INFO    ] 2022-09-26 11:26:21 | LocationUpdater: 6042 addresses in index
[INFO    ] 2022-09-26 11:26:21 | LocationUpdater: 6081 addresses total
[INFO    ] 2022-09-26 11:26:21 | LocationUpdater: 6042 addresses defined
[INFO    ] 2022-09-26 11:26:21 | LocationUpdater: 39 addresses undefined


  0%|          | 0/39 [00:00<?, ?it/s]

[INFO    ] 2022-09-26 11:26:33 | LocationUpdater: 39 new addresses found


In [31]:
print( len(loc) )
loc.sample(3)

6081


,latitude,longitude,adr
1253,44.60,33.45,"Севастополь, район Гагаринский, улица Летчиков, 6"
915,44.61,33.52,"Севастополь,Советская улица , 24"
2219,44.596254,33.457331,"Севастополь, район Гагаринский, Летчики, проспект Октябрьской Революции, 23 к3"


## дополняем данные геометкой

In [32]:
# data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

In [33]:
data = data.merge(loc,on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isna() ] ) )

всего записей: 32214
записей без геометки: 0


In [34]:
assert len(data)>0
data.to_pickle(result_file_path)

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32214 entries, 0 to 32213
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          32214 non-null  object        
 1   price          32214 non-null  int64         
 2   adr_orig       32214 non-null  object        
 3   description    32214 non-null  object        
 4   obj_name       32214 non-null  object        
 5   ts             32214 non-null  datetime64[ns]
 6   avito_page     20271 non-null  float64       
 7   place          32214 non-null  object        
 8   nrooms         32214 non-null  int64         
 9   floor          32214 non-null  int64         
 10  nfloors        32214 non-null  int64         
 11  area           32214 non-null  float64       
 12  is_studio      32214 non-null  bool          
 13  is_apartment   32214 non-null  bool          
 14  is_part        32214 non-null  bool          
 15  is_auction     3221